In [1]:
### Imports

import matplotlib.pyplot as plt
plt.rc('font',family='AppleGothic')
import numpy as np
import pandas as pd
# Load_data

data_raw = pd.read_csv('../used/data/raw_data/train.csv')
sample_submission_raw = pd.read_csv('../used/data/raw_data/sample_submission.csv')

data=data_raw.copy()
sample_submission=sample_submission_raw.copy()

# Data_index to timeseries

data.set_index('일시',inplace=True)
data.index=pd.to_datetime(data.index,format='%Y%m%d')
data['week']=data.index.isocalendar().week
data['day_name']=data.index.weekday
data['weekend']=data['day_name'].apply(lambda x: 0 if x<5 else 1)
data['year']=data.index.year

## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## 

sample_submission.set_index('일시',inplace=True)
sample_submission.index=pd.to_datetime(sample_submission.index,format='%Y%m%d')
sample_submission['week']=sample_submission.index.isocalendar().week
sample_submission['day_name']=sample_submission.index.weekday
sample_submission['weekend']=sample_submission['day_name'].apply(lambda x: 0 if x<5 else 1)
sample_submission['year']=sample_submission.index.year
# 년도별 공휴일 체크하기위한 library => 평일인 공휴일도 주말로 설정.

from pytimekr import pytimekr
def get_holiday(_year):
    holidays=[]
    for holiday in pytimekr.holidays(year=_year):
        if pytimekr.red_days(holiday) != None:
            ans = [i.strftime("%Y-%m-%d") for i in pytimekr.red_days(holiday)]
            holidays.extend(ans)
        else:
            ans=holiday.strftime("%Y-%m-%d")
            holidays.append(ans)
    return list(set(holidays))

data.loc[get_holiday(2018),'weekend'] = 1
data.loc[get_holiday(2019),'weekend'] = 1
data.loc[get_holiday(2020),'weekend'] = 1
data.loc[get_holiday(2021),'weekend'] = 1
sample_submission.loc[sorted(get_holiday(2022))[:-1],'주말평일'] ='주말'
data

,광진구,동대문구,성동구,중랑구,week,day_name,weekend,year
일시,,,,,,,,
2018-01-01,0.592,0.368,0.580,0.162,1,0,1,2018
2018-01-02,0.840,0.614,1.034,0.260,1,1,0,2018
2018-01-03,0.828,0.576,0.952,0.288,1,2,0,2018
2018-01-04,0.792,0.542,0.914,0.292,1,3,0,2018
2018-01-05,0.818,0.602,0.994,0.308,1,4,0,2018
...,...,...,...,...,...,...,...,...
2021-12-27,3.830,3.416,2.908,2.350,52,0,0,2021
2021-12-28,4.510,3.890,3.714,2.700,52,1,0,2021
2021-12-29,4.490,3.524,3.660,2.524,52,2,0,2021


In [4]:
temp=data.loc[:,['광진구','weekend']]

In [7]:
temp=temp.reset_index()

In [11]:
temp

,일시,광진구,weekend
0,2018-01-01,0.592,1
1,2018-01-02,0.840,0
2,2018-01-03,0.828,0
3,2018-01-04,0.792,0
4,2018-01-05,0.818,0
...,...,...,...
1456,2021-12-27,3.830,0
1457,2021-12-28,4.510,0
1458,2021-12-29,4.490,0
1459,2021-12-30,4.444,0


In [1]:
from neuralprophet import NeuralProphet

In [2]:
NeuralProphet.add_country_holidays(country_name='korea')

TypeError: add_country_holidays() missing 1 required positional argument: 'self'

In [ ]:
NeuralProphet.add_events

In [9]:
temp.value_counts('weekend')

weekend
0    1003
1     458
dtype: int64

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth
# 년도별 주별 (평일,주말) 별 평균값 구하기
data_week_groupby=data.groupby(['year','week','weekend']).mean().reset_index()